In [1]:
import mne
import mne_bids
from mne_bids import BIDSPath
import os, os.path as op
import nilearn
import matplotlib.pyplot as plt
import numpy as np
from mne.minimum_norm import apply_inverse, make_inverse_operator

from nilearn import *

from nih2mne.utilities.bids_helpers import get_mri_dict
n_jobs = 10 #Number of parrallel operations

## Set some generic BIDS information

In [2]:
#Raw data
bids_root = op.join('/data/', os.environ['USER'], 'meg_workshop_data')  
#Processed Data Folder: contains (freesurfer / Day1 / Day2) 
deriv_root = op.join(bids_root, 'derivatives')  
#This course data - bids derivatives outputs for Day2 - hey thats today
project_root = op.join(deriv_root, 'Day2')  
#Freesurfer brain surface reconstruction
fs_subjects_dir = subjects_dir = op.join(deriv_root, 'freesurfer','subjects')

### Find data for a subject

In [7]:
subject = 'ON02811'
data_dict = get_mri_dict(subject,bids_root, task='airpuff', project='Day2')

#Not in BIDs format
stc_fname = op.join(project_root, 'ses-01', 'meg', 'sub-ON02811_ses-01_task-airpuff_run-01_desc-missingstim_stc30to50-vl.stc')

# get_mri_dict is a helper function that provides all of the loaders for the mri integration
# Each dictionary item codes for the specific loader of that file - see use in next cell of code
data_dict

{'bem': <nih2mne.utilities.bids_helpers.data_getter at 0x1554bd92bd10>,
 'fwd': <nih2mne.utilities.bids_helpers.data_getter at 0x155551702910>,
 'src': <nih2mne.utilities.bids_helpers.data_getter at 0x1554bfd30350>,
 'trans': <nih2mne.utilities.bids_helpers.data_getter at 0x1554bda52b50>,
 'volfwd': <nih2mne.utilities.bids_helpers.data_getter at 0x1554bda53790>}

#### Load the preprocessed data - Items from Lab3 MRI Processing

In [9]:
bem = data_dict['bem'].load()
fwd = data_dict['fwd'].load()
volfwd = data_dict['volfwd'].load()
src = fwd['src']
trans = data_dict['trans'].load()

Loading surfaces...

Loading the solution matrix...

Homogeneous model surface loaded.
Loaded linear collocation BEM solution from /data/stoutjd/meg_workshop_data/derivatives/Day2/sub-ON02811/ses-01/meg/sub-ON02811_ses-01_task-airpuff_run-01_bem.fif
Reading forward solution from /data/stoutjd/meg_workshop_data/derivatives/Day2/sub-ON02811/ses-01/meg/sub-ON02811_ses-01_task-airpuff_run-01_fwd.fif...
    Reading a source space...
    Computing patch statistics...
    Patch information added...
    [done]
    Reading a source space...
    Computing patch statistics...
    Patch information added...
    [done]
    2 source spaces read
    Desired named matrix (kind = 3523) not available
    Read MEG forward solution (8196 sources, 272 channels, free orientations)
    Source spaces transformed to the forward solution coordinate frame
Reading forward solution from /data/stoutjd/meg_workshop_data/derivatives/Day2/sub-ON02811/ses-01/meg/sub-ON02811_ses-01_task-airpuff_run-01_volfwd.fif...
    

/vf/users/MEGmodules/modules/meg_workshop_2023_extras/nih_to_mne/nih2mne/utilities/bids_helpers.py:109: RuntimeWarning: This filename (/data/stoutjd/meg_workshop_data/derivatives/Day2/sub-ON02811/ses-01/meg/sub-ON02811_ses-01_task-airpuff_run-01_volfwd.fif) does not conform to MNE naming conventions. All forward files should end with -fwd.fif, -fwd.fif.gz, _fwd.fif or _fwd.fif.gz
  return self.loader(self.bids_path.fpath)


    [done]
    1 source spaces read
    Desired named matrix (kind = 3523) not available
    Read MEG forward solution (9593 sources, 272 channels, free orientations)
    Source spaces transformed to the forward solution coordinate frame


In [5]:
#Load the Raw data
bids_path = BIDSPath(root=bids_root, subject=subject, task='airpuff', run='01',session='01')
raw = mne.io.read_raw_ctf(bids_path.fpath, clean_names=True, preload=True, verbose=False)
raw.filter(0.3, 110, n_jobs=n_jobs)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.3 - 1.1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.30
- Lower transition bandwidth: 0.30 Hz (-6 dB cutoff frequency: 0.15 Hz)
- Upper passband edge: 110.00 Hz
- Upper transition bandwidth: 27.50 Hz (-6 dB cutoff frequency: 123.75 Hz)
- Filter length: 13201 samples (11.001 s)



[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   8 tasks      | elapsed:    3.6s
[Parallel(n_jobs=10)]: Done 116 tasks      | elapsed:    4.6s
[Parallel(n_jobs=10)]: Done 300 out of 300 | elapsed:    6.0s finished


<RawCTF | sub-ON02811_ses-01_task-airpuff_run-01_meg.meg4, 304 x 301200 (251.0 s), ~699.0 MB, data loaded>

### Prepare Imaging for Source Localization

In [11]:
volfwd

Good channels,272 Magnetometers
Bad channels,None
Source space,Volume with 9593 grid points
Source orientation,Free


In [12]:
fsave_src = op.join(fs_subjects_dir, 'fsaverage', 'bem','fsaverage-ico-5-src.fif')

morph = mne.compute_source_morph(volfwd['src'], subject_from = 'sub-'+subject, 
                                 subject_to='fsaverage', subjects_dir=fs_subjects_dir)

Volume source space(s) present...
    Loading /data/stoutjd/meg_workshop_data/derivatives/freesurfer/subjects/sub-ON02811/mri/brain.mgz as "from" volume
    Loading /data/stoutjd/meg_workshop_data/derivatives/freesurfer/subjects/fsaverage/mri/brain.mgz as "to" volume
Computing registration...
Reslicing to zooms=(5.0, 5.0, 5.0) for translation ...
Optimizing translation:
    Optimizing level 2 [max iter: 100]
    Optimizing level 1 [max iter: 100]
    Optimizing level 0 [max iter: 10]
    Translation:   33.4 mm
    R²:            85.6%
Optimizing rigid:
    Optimizing level 2 [max iter: 100]
    Optimizing level 1 [max iter: 100]
    Optimizing level 0 [max iter: 10]
    Translation:   33.4 mm
    Rotation:      15.9°
    R²:            86.9%
Optimizing affine:
    Optimizing level 2 [max iter: 100]
    Optimizing level 1 [max iter: 100]
    Optimizing level 0 [max iter: 10]
    R²:            96.4%
Optimizing sdr:
    R²:            98.5%
Computing nonzero vertices after morph ...
[don

In [13]:
morph

<SourceMorph | volume, sub-ON02811 -> fsaverage, zooms : (5.0, 5.0, 5.0), niter_affine : (100, 100, 10), niter_sdr : (5, 5, 3)>

In [15]:
morph.morph_mat

In [17]:
morph.affine

array([[  -5.,    0.,    0.,  128.],
       [   0.,    0.,    5., -128.],
       [   0.,   -5.,    0.,  128.],
       [   0.,    0.,    0.,    1.]])

In [18]:
morph.morph_mat

In [22]:
morph.sdr_morph.get_backward_field().shape

(51, 51, 51, 3)

In [31]:

tmp=volfwd['src'][0]

In [32]:
tmp

{'id': 101,
 'type': 'vol',
 'shape': (31, 35, 29),
 'src_mri_t': <Transform | MRI voxel->MRI (surface RAS)>
 [[ 0.005  0.     0.    -0.08 ]
  [ 0.     0.005  0.    -0.1  ]
  [ 0.     0.     0.005 -0.06 ]
  [ 0.     0.     0.     1.   ]],
 'mri_ras_t': <Transform | MRI (surface RAS)->RAS (non-zero origin)>
 [[ 1.          0.          0.          0.002055  ]
  [ 0.          1.          0.          0.03346001]
  [ 0.          0.          1.         -0.01727301]
  [ 0.          0.          0.          1.        ]],
 'vox_mri_t': <Transform | MRI voxel->MRI (surface RAS)>
 [[-0.001       0.          0.          0.12800001]
  [ 0.          0.          0.001      -0.12800001]
  [ 0.         -0.001       0.          0.12800001]
  [ 0.          0.          0.          1.        ]],
 'interpolator': <16777216x31465 sparse matrix of type '<class 'numpy.float32'>'
 	with 8227720 stored elements in Compressed Sparse Row format>,
 'mri_width': 256,
 'mri_height': 256,
 'mri_depth': 256,
 'mri_volum

In [39]:

tmp['nn'].shape

(31465, 3)

In [41]:
tmp['shape']


(31, 35, 29)

In [42]:
31*35*29

31465